In [37]:
import pandas as pd

In [38]:
calibration_df = pd.read_csv("calibration.csv", delimiter=";")
calibration_df

,home_xg_calibration,away_xg_calibration,home_goals_calibration,away_goals_calibration
0,0.855493,0.934626,0.941699,0.872308


In [39]:
log_loss_df = pd.read_csv("log_loss.csv", delimiter=";")
log_loss_df

,venomio_log_loss_home_win,venomio_log_loss_away_win,venomio_log_loss_draw_win,venomio_log_loss_overunder_25,market_log_loss_home_win,market_log_loss_away_win,market_log_loss_draw_win,market_log_loss_overunder_25
0,0.630381,0.584948,0.550285,0.671031,0.632407,0.56827,0.54635,0.600276


In [40]:
rmse_df = pd.read_csv("rmse.csv", delimiter=";")
rmse_df

,rmse_home_xg,rmse_away_xg,rmse_home_goals,rmse_away_goals,benchmark_rmse_home_goals,benchmark_rmse_away_goals
0,0.840257,0.721796,1.16976,1.118426,1.042203,0.891251


In [41]:
betting_df = pd.read_excel("betting_results.xlsx")
betting_df

,Match,Date,Market,Odds,Market Odds,EV,Result,Bet Amount (%),Profit/Loss (%),minutes_strength
0,Charlotte vs Chicago Fire,2024-10-02,draw,3.569,4.219,0.182124,-1.0,0.025802,-0.025802,0.912100
1,Charlotte vs Chicago Fire,2024-10-02,away,3.704,5.269,0.422516,-1.0,0.045137,-0.045137,0.912100
2,Charlotte vs Chicago Fire,2024-10-02,under_15,3.700,4.560,0.232432,-1.0,0.029776,-0.029776,0.912100
3,Charlotte vs Chicago Fire,2024-10-02,under_25,1.921,2.280,0.186882,-1.0,0.066584,-0.066584,0.912100
4,Charlotte vs Chicago Fire,2024-10-02,under_35,1.368,1.510,0.103801,-1.0,0.072968,-0.072968,0.912100
...,...,...,...,...,...,...,...,...,...,...
3041,Espanyol vs Celta Vigo,2024-11-30,home_0,2.301,2.610,0.134289,1.0,0.033826,0.054460,0.811087
3042,Espanyol vs Celta Vigo,2024-11-30,home_p025,1.850,2.000,0.081081,1.0,0.032882,0.032882,0.811087
3043,Espanyol vs Celta Vigo,2024-11-30,home_m025,2.832,3.070,0.084040,1.0,0.016465,0.034082,0.811087
3044,Espanyol vs Celta Vigo,2024-11-30,home_p075,1.453,1.520,0.046111,1.0,0.035962,0.018700,0.811087


In [42]:
results_df = pd.DataFrame(betting_df)

# Convert numeric columns to proper types
results_df['Market Odds'] = pd.to_numeric(results_df['Market Odds'], errors='coerce')
results_df['Odds'] = pd.to_numeric(results_df['Odds'], errors='coerce')
results_df['EV'] = pd.to_numeric(results_df['EV'], errors='coerce')
results_df['Result'] = pd.to_numeric(results_df['Result'], errors='coerce')
results_df['Bet Amount (%)'] = pd.to_numeric(results_df['Bet Amount (%)'], errors='coerce')
results_df['Profit/Loss (%)'] = pd.to_numeric(results_df['Profit/Loss (%)'], errors='coerce')

column_order = ['Match', 'Date', 'Market', 'Odds', 'Market Odds', 'EV', 
                'Result', 'Bet Amount (%)', 'Profit/Loss (%)', 'minutes_strength']
results_df = results_df[column_order]

results_df.to_excel('betting_results.xlsx', index=False)

print(f"Betting results saved to 'betting_results.xlsx'")

# Calculate statistics by market
print(f"\n--- Statistics by Market Type ---")

# Group by market and calculate statistics
market_stats = results_df.groupby('Market').agg({
    'Market Odds': 'mean',
    'Result': ['count', lambda x: (x > 0).sum(), lambda x: (x < 0).sum()],
    'Profit/Loss (%)': 'sum',
    'Bet Amount (%)': 'mean'
}).round(2)

# Flatten column names
market_stats.columns = ['Avg Market Odds', 'Total Bets', 'Winning Bets', 'Losing Bets', 'Total P/L (%)', 'Avg Bet Amount (%)']

# Calculate win rate and average winning market odds
market_stats['Win Rate (%)'] = (market_stats['Winning Bets'] / market_stats['Total Bets'] * 100).round(2)

# Calculate average winning market odds (only for winning bets)
winning_bets_stats = results_df[results_df['Result'] > 0].groupby('Market')['Market Odds'].mean().round(2)
winning_bets_stats = winning_bets_stats.fillna(0)  # Handle cases with no winning bets
market_stats['Avg Winning Market Odds'] = winning_bets_stats

# Reorder columns for better readability
market_stats = market_stats[['Total Bets', 'Winning Bets', 'Losing Bets', 'Win Rate (%)', 
                            'Total P/L (%)', 'Avg Bet Amount (%)', 'Avg Market Odds', 'Avg Winning Market Odds']]

print(market_stats)

# Overall statistics
total_bets = len(results_df)
winning_bets = len(results_df[results_df['Result'] > 0])
losing_bets = len(results_df[results_df['Result'] < 0])
push_bets = len(results_df[results_df['Result'] == 0])
total_pl = results_df['Profit/Loss (%)'].sum()
win_rate = (winning_bets / total_bets) * 100 if total_bets > 0 else 0
avg_market_odds = results_df['Market Odds'].mean()
avg_bet_amount = results_df['Bet Amount (%)'].mean()

print(f"\n--- Overall Statistics ---")
print(f"Total bets placed: {total_bets}")
print(f"Winning bets: {winning_bets}")
print(f"Push bets: {push_bets}")
print(f"Losing bets: {losing_bets}")
print(f"Win rate: {win_rate:.2f}%")
print(f"Total P/L: {total_pl:.2f}%")
print(f"Average market odds: {avg_market_odds:.2f}")
print(f"Average bet amount: {avg_bet_amount:.2f}%")

Betting results saved to 'betting_results.xlsx'

--- Statistics by Market Type ---
           Total Bets  Winning Bets  Losing Bets  Win Rate (%)  Total P/L (%)  \
Market                                                                          
away              138            39           99         28.26           0.99   
away_0            150            45           64         30.00           0.83   
away_m025         122            40           32         32.79           2.88   
away_m075         109            37           72         33.94           0.76   
away_p025         154            86           68         55.84           0.91   
away_p075         159            85           74         53.46           1.29   
draw              161            36          125         22.36           0.04   
home               76            29           47         38.16           0.40   
home_0             84            31           34         36.90           0.13   
home_m025          77     